<a href="https://colab.research.google.com/github/CarolineAndradeR/Python/blob/master/Regress%C3%A3o_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conceito**



**Definição** - O modelo de regressão busca ajustar a melhor reta em uma nuvem de pontos. Quando o objetivo for analisar/prever saídas utilizado variáveis quantitativas (renda, peso) a técnica de regressão deve ser utilizada. A regressão busca minimizar os erros.


**Regressão Simples** - Uma variavel para projetar uma outra variavel.

**Regressão Múltipla** - Divesas variaveis para projetar uma outra.

---
Termos utilizados.

**Machine Learning**------**Estatística**---------------------------**Reconhecimento de Padrões**     
Inputs-------------------------Variáveis Independetes----------------Features

Feature Engineering---Transformação de Variáveis--------Feature engineering

Outputs-----------------------Variáveis Dependentes---------------Outputs

Amostra de Treino------Análise in-the-sample-----------------Amostra de Treino

Amostra de Teste-------Análise out-of-sample-----------------Amostra de Treino

---

**Outlier -** Quando se tem um ponto totalmente fora do padrão.
**RMSE -** É utilizada para validar modelos supervisionados com objetivos de regressão. Quanto maior o RMSE, maior o erro e com isso pior é o modelo.

# **Pratica**



In [1]:
# Modulos gerias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Modulos de análise de regressão
# Treinar o modelo e testar em outro conjunto de dados
from sklearn.model_selection import train_test_split
# Algoritimo de Regressão
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
# Medida para verificar se o modelo funcionou bem
from sklearn.metrics import mean_squared_error

In [2]:
#Iniciando montagem do arquivo com o drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

dados_gastos_cartao = '/content/drive/MyDrive/Asset_FIAP/base_gastos_cartao.csv'
dados_gastos_cartao = pd.read_csv(dados_gastos_cartao)
dados_gastos_cartao.head()

Mounted at /content/drive


,gastos_cartao,idade,renda,impostos,segmento
0,510,35,1120,60,C
1,490,30,1120,60,C
2,470,32,1040,60,C
3,460,31,1200,60,C
4,500,36,1120,60,C


In [3]:
# Dados treino e teste
# Test_size - valor da divsão para teste
# 34251 - Manter a divisão dos valores
train_data, test_data = train_test_split(dados_gastos_cartao,
                                         test_size = 0.33,
                                         random_state = 34251)
# numero de linhas para cada dataset
dados_gastos_cartao.shape[0], train_data.shape[0], test_data.shape[0]

(150, 100, 50)

In [4]:
# Treino
# Uma única variavel de renda (regressão linear simples)
# Verificar se os gastos de cartão é explicado pela renda da pessoa
X = train_data['renda'].values.reshape(-1, 1)
y = train_data['gastos_cartao']

# Teste
X_test = test_data['renda'].values.reshape(-1, 1)
y_test = test_data['gastos_cartao']

# Treinar o modelo
modelo_1 = LinearRegression().fit(X, y)

In [5]:
# Coeficiente e o valor de beta, um valor medio e o valor do coeficiente 
modelo_1.intercept_, modelo_1.coef_[0]
# r2 verifica se o modelo ficou bem ajustado com os dados
modelo_1.score(X, y)

0.7626015444308568

In [6]:
# Apó o modelo treinado, podemos usar os dados para realizar previsões
# Quanto a pessoa pode gastar com base na sua renda
y_pred = modelo_1.predict(X_test)

# Avaliar o valor da diferença entre o y real (teste) e o valor de y previsto, quanto menor o valor melhor siginfica que o modelo está acertando mais.
# Esse valor sozino não traz tanto resutados, mas quando combinados com outras analises de RMSE para verificar qual teve o menor valor de RMSE.
# mean_squared_error - Erro medio quadrado
# sqrt - valor em reais
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

43.439816006926726

In [7]:
# sm.ols - função de regressão linear
# Nessa função podemos trabalhar diretamente com as colunas

# modelo regressoa statsmodels
modelo_1_sm = sm.ols(formula = 'gastos_cartao ~ renda',
                     data = train_data).fit()
# modelo regressoa statsmodels
modelo_1_sm.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.760    
Dependent Variable: gastos_cartao    AIC:                1020.9293
Date:               2023-05-27 19:41 BIC:                1026.1396
No. Observations:   100              Log-Likelihood:     -508.46  
Df Model:           1                F-statistic:        314.8    
Df Residuals:       98               Prob (F-statistic): 2.30e-32 
R-squared:          0.763            Scale:              1558.7   
-------------------------------------------------------------------
             Coef.    Std.Err.     t     P>|t|    [0.025    0.975] 
-------------------------------------------------------------------
Intercept   428.1370    9.6464  44.3831  0.0000  408.9941  447.2800
renda         0.0508    0.0029  17.7428  0.0000    0.0451    0.0565
------------------------------------------------------------------
Omnibus:              0.402         Durbin-Watson:           1.886
Prob(Omnibus):        0.818         Jarque-Bera (JB):        0.514
Skew:                 -0.139        Prob(JB):                0.773
Kurtosis:             2.786         Condition No.:           8233 
==================================================================
* The condition number is large (8e+03). This might indicate
strong multicollinearity or other numerical problems.
"""

In [8]:
# Avaliando a amostra de teste
# previsao
y_pred = modelo_1_sm.predict(test_data)
 
# rmse
rmse = np.sqrt(mean_squared_error(test_data['gastos_cartao'], y_pred))
rmse

43.439816006926755

Regressão Multipla

In [12]:
# Análise de regressão multipla com todos os campos da tabela
# Antes precisamos mudar as colunas do tipo texto para variaveis do tipo dummies

# Vai ser criada 3 novas colunas, colunas A B C, na nova coluna A o cliente que for segmentado com A recebe 1 e na coluna B o segmentado com B recebe 1 e os outros clientes e categorizados com 0
# Só é feito isso com as 2 primeiras, porque na terceira recebe o valor da diferença entre as duas primeiras.
# Como boa pratica de processamento precisamos dar drop em um desses grupos, nesse caso o grupo sera o A.

# variavel segmento é categorica, para ajustar
segmento_data_train = pd.get_dummies(data=train_data['segmento'], drop_first=True)
segmento_data_test = pd.get_dummies(data=test_data['segmento'], drop_first=True)

# dados transformados
segmento_data_test.head()

,B,C
135,0,0
37,0,1
141,0,0
131,0,0
137,0,0


In [14]:
# Separando as variaveis de treino
cols_reg = ['idade', 'renda', 'impostos']
X = train_data[cols_reg]
# Concatenando o valores de X de cima e segmento formado.
X = pd.concat([X, segmento_data_train], axis=1)
y = train_data['gastos_cartao']
 
# Separando as variaveis de teste
X_test = test_data[cols_reg]
X_test = pd.concat([X_test, segmento_data_test], axis=1)
y_test = test_data['gastos_cartao']

In [15]:
# treino
modelo_2 = LinearRegression().fit(X, y)

In [17]:
# Avaliação da amostra de treino
# coeficientes
betas = np.append(modelo_2.intercept_, modelo_2.coef_)
colunas = ['intercept']+X.columns.to_list()

# Quanto mais a idede mais gasto
# Imposto negativo quanto mais a pessoa paga imposto menos ela gasta no cartão
# O grupo C gasta mais que o grupo A e B
coeficientes = pd.DataFrame(betas, colunas, columns=['Coeficientes'])
coeficientes


,Coeficientes
intercept,113.116933
idade,4.708346
renda,0.104301
impostos,-0.102207
B,37.276483
C,109.457414


In [18]:
# Coeficiente de determinação
r2 = modelo_2.score(X, y)
r2

0.8529140318990082

In [19]:
# Amostra de teste
# previsao
y_pred = modelo_2.predict(X_test)
# rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse
# O modelo está errando em torno de 29$ do verdadeiro gasto do cartão

29.142266174570555

In [21]:
# Analisando com Statsmodels
# C(segmento) - Transforma em Dumminus automaticamente
modelo_2_sm = sm.ols(formula = 'gastos_cartao ~ idade + renda + impostos + C(segmento)',
                     data = train_data).fit()
# coeficientes
modelo_2_sm.summary2()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:               OLS              Adj. R-squared:     0.845   
Dependent Variable:  gastos_cartao    AIC:                981.0570
Date:                2023-05-27 20:14 BIC:                996.6880
No. Observations:    100              Log-Likelihood:     -484.53 
Df Model:            5                F-statistic:        109.0   
Df Residuals:        94               Prob (F-statistic): 1.50e-37
R-squared:           0.853            Scale:              1006.8  
------------------------------------------------------------------
                  Coef.   Std.Err.    t    P>|t|   [0.025  0.975] 
------------------------------------------------------------------
Intercept        113.1169  43.6690  2.5903 0.0111 26.4112 199.8227
C(segmento)[T.B]  37.2765  15.0123  2.4831 0.0148  7.4693  67.0837
C(segmento)[T.C] 109.4574  43.9326  2.4915 0.0145 22.2283 196.6865
idade              4.7083   1.4094  3.3406 0.0012  1.9099   7.5068
renda              0.1043   0.0104 10.0376 0.0000  0.0837   0.1249
impostos          -0.1022   0.0731 -1.3980 0.1654 -0.2474   0.0430
------------------------------------------------------------------
Omnibus:              4.590         Durbin-Watson:           1.926
Prob(Omnibus):        0.101         Jarque-Bera (JB):        3.440
Skew:                 -0.321        Prob(JB):                0.179
Kurtosis:             2.357         Condition No.:           58036
==================================================================
* The condition number is large (6e+04). This might indicate
strong multicollinearity or other numerical problems.
"""

In [22]:
# Avaliação da amostra de teste
# previsao
y_pred = modelo_2_sm.predict(test_data)
# rmse
rmse = np.sqrt(mean_squared_error(test_data['gastos_cartao'], y_pred))
rmse# previsao
y_pred = modelo_2_sm.predict(test_data)
# rmse
rmse = np.sqrt(mean_squared_error(test_data['gastos_cartao'], y_pred))
rmse

29.142266174573628